In [ ]:
# Mounting drive
from google.colab import drive, files
drive.mount('/content/drive')

files.upload() #this will prompt you to upload the kaggle.json
# after uploading the .json file you will get your kaggle username and key as output

In [ ]:
!pip3 install kaggle

%env KAGGLE_USERNAME = ""  # enter username and key without "" (i.e. remove "")
%env KAGGLE_KEY = ""   

# go to your dataset and click the "copy API command" and paste below with an "!" in the front-

# replace "filename" with your downloaded filename
!unzip File Name

In [ ]:
# Importing stuff
import numpy as np
from glob import glob
from matplotlib import pyplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Defining some global variable
IMG_H = 64
IMG_W = 64
IMG_C = 3
latent_dim = 128 # hyper parameter
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02) # weight initialization 

In [ ]:
#########################################################################################################################################################
#TODO write the load_image() that takes image_path as input and resizes the image to 64x64x3 with every pixel normalized to (-1, 1), dont forget to convert the image to tf.float32
# Use only tensorflow to process the image                                                                       
def load_image(image_path):
  
    return img
# funtion for making dataset
def tf_dataset(images_path, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(images_path)
    dataset = dataset.shuffle(buffer_size=10240)
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
###########################################################################################################################################################
################################################################## Write your discriminator ###############################################################

discriminator = keras.Sequential([
    ], name = "discriminator"
    )
discriminator.summary()

In [ ]:
###########################################################################################################################################################
################################################################## Write your generator ###################################################################

generator = keras.Sequential([
    ], name = "generator"
    )
generator.summary()

In [ ]:
#defining GAN class with modified train_step function.
class GAN(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]


            ## Train the discriminator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)
        generated_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as ftape:
            predictions = self.discriminator(generated_images)
            d1_loss = self.loss_fn(generated_labels, predictions)
        grads = ftape.gradient(d1_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
            ## Train the discriminator
        labels = tf.ones((batch_size, 1))

        with tf.GradientTape() as rtape:
            predictions = self.discriminator(real_images)
            d2_loss = self.loss_fn(labels, predictions)
        grads = rtape.gradient(d2_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        ## Train the generator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.ones((batch_size, 1))

        with tf.GradientTape() as gtape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = gtape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d1_loss": d1_loss, "d2_loss": d2_loss, "g_loss": g_loss}

In [ ]:
# function for saving plots 
def save_plot(examples, epoch, n):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        pyplot.subplot(n, n, i+1)
        pyplot.axis("off")
        pyplot.imshow(examples[i])  
    filename = f"samples/generated_plot_epoch-{epoch+1}.png"  # edit this as you like just make sure "samples"  folder exists in the current working directory
    pyplot.savefig(filename)
    pyplot.close()

In [ ]:
# main function containing all funtcion calls and hyper paramerters


if __name__ == "__main__":
    ## Hyperparameters
    batch_size = 128
    num_epochs = 80
    images_path = glob("archive/images/*") #image path
    
    #discriminator.load_weights("saved_model/d_model.h5")
    #generator.load_weights("saved_model/g_model.h5")
    
    gan = GAN(discriminator, generator, latent_dim)
# loss function and optimizers
    bce_loss_fn =   # TODO define a loss function
    d_optimizer =   # TODO define discriminator optimizer
    g_optimizer =   # TODO define generator optimizer
# Compiling
    gan.compile(d_optimizer, g_optimizer, bce_loss_fn)

    images_dataset = tf_dataset(images_path, batch_size)
# training
    for epoch in range(num_epochs):
        gan.fit(images_dataset, epochs=1)
        generator.save("saved_model/g_model.h5")
        discriminator.save("saved_model/d_model.h5")

        n_samples = 25
        noise = np.random.normal(size=(n_samples, latent_dim))
        examples = generator.predict(noise)
        save_plot(examples, epoch, int(np.sqrt(n_samples)))